# This notebook calculates the personal preparedness metrics sourced from FEMA
* % of survey respondents who can live in their homes for at least 3 days without running water
* % of survey respondents who can live in their homes for at least 3 days without power
* % of survey respondents who have engaged in at least one preparedness action in the last year

In [1]:
import pandas as pd
import os
import sys
import math
import numpy as np

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_csv_from_directory, upload_csv_aws, filter_counties
from scripts.utils.write_metadata import append_metadata

In [2]:
# pull csv from aws
bucket_name = 'ca-climate-index'
aws_dir = '2a_subset/governance/personal_preparedness/fema/fema_national_household_survey/'

pull_csv_from_directory(bucket_name, aws_dir, search_zipped=False)

Saved DataFrame as 'fema_prepareness_survey_subset.csv'


In [3]:
fema_household_data = pd.read_csv('fema_prepareness_survey_subset.csv')

In [4]:
fema_household_data

,Unique Respondent ID (IDs are consistent between NHS General Data and NHs Hazards Data. Respondents within the hazard dataset have the same IDs in the general dataset).,Oversample Hazard type,General Sample Weight,Phone/Online,Online/Cellphone/Landline Phone,What is the name of the state or territory you live in?,Census Division,Census Region,What is your ZIP Code?,What county in [state] do you live in?,...,"Generated column - rurality calculated using ZIP code, county, and state",Demographics imputed (Yes/No),Demographics imputed (Yes/No).1,Demographics imputed (Yes/No).2,Demographics imputed (Yes/No).3,Demographics imputed (Yes/No).4,Demographics imputed (Yes/No).5,Demographics imputed (Yes/No).6,Demographics imputed (Yes/No).7,Language in which survey was completed
0,6,Active Shooter,1.932171,Online,Online,California,Pacific,West,94550,Alameda,...,Urban,No,No,No,No,No,No,No,No,English
1,8,Active Shooter,1.066004,Online,Online,California,Pacific,West,95841,Sacramento,...,Urban,No,No,No,No,No,No,No,No,English
2,11,Active Shooter,3.601788,Online,Online,California,Pacific,West,92708,Orange,...,Urban,No,No,No,No,No,No,No,No,English
3,14,Active Shooter,0.464613,Online,Online,California,Pacific,West,90028,Los Angeles,...,Urban,No,No,No,No,No,No,No,No,English
4,15,Active Shooter,0.818878,Online,Online,California,Pacific,West,95835,Sacramento,...,Urban,No,No,No,No,No,No,No,No,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
711,7119,NaN,1.841526,Phone,Landline phone,California,Pacific,West,95687,Solano,...,Urban,No,No,No,No,No,No,No,No,English
712,7130,NaN,0.618372,Phone,Landline phone,California,Pacific,West,90405,Los Angeles,...,Urban,No,No,No,No,No,No,No,No,English
713,7132,NaN,0.660724,Phone,Landline phone,California,Pacific,West,90403,Los Angeles,...,Urban,No,No,No,No,No,No,No,No,English
714,7133,NaN,1.772459,Phone,Landline phone,California,Pacific,West,92346,San Bernardino,...,Urban,No,No,No,No,No,No,No,No,English


In [8]:
pd.set_option('display.max_columns', None)
print(fema_household_data.columns)

Index(['Unique Respondent ID (IDs are consistent between NHS General Data and NHs Hazards Data. Respondents within the hazard dataset have the same IDs in the general dataset).',
       'Oversample Hazard type', 'General Sample Weight', 'Phone/Online',
       'Online/Cellphone/Landline Phone',
       'What is the name of the state or territory you live in?',
       'Census Division', 'Census Region', 'What is your ZIP Code?',
       'What county in [state] do you live in? ',
       ...
       'Generated column - rurality calculated using ZIP code, county, and state',
       'Demographics imputed (Yes/No)', 'Demographics imputed (Yes/No).1',
       'Demographics imputed (Yes/No).2', 'Demographics imputed (Yes/No).3',
       'Demographics imputed (Yes/No).4', 'Demographics imputed (Yes/No).5',
       'Demographics imputed (Yes/No).6', 'Demographics imputed (Yes/No).7',
       'Language in which survey was completed'],
      dtype='object', length=368)


In [14]:
selected_columns = ['What county in [state] do you live in? ',
'How long could you live in your home without power?',
'How long could you live in your home without running water? ',
'Thinking about preparing yourself for a disaster, which of the following best represents your degree of preparedness? ']

selected_fema_household_data = fema_household_data[selected_columns]
selected_fema_household_data

,What county in [state] do you live in?,How long could you live in your home without power?,How long could you live in your home without running water?,"Thinking about preparing yourself for a disaster, which of the following best represents your degree of preparedness?"
0,Alameda,3 days to 1 week,Don't know,"I am NOT prepared, but I intend to get prepare..."
1,Sacramento,1 to 3 days,1 to 3 days,"I am NOT prepared, and I do not intend to prep..."
2,Orange,Don't know,3 days to 1 week,I have been prepared for MORE than a year and ...
3,Los Angeles,3 days to 1 week,3 days to 1 week,"I am NOT prepared, but I intend to get prepare..."
4,Sacramento,3 days to 1 week,1 to 3 days,"I am NOT prepared, but I intend to get prepare..."
...,...,...,...,...
711,Solano,More than 1 month,More than 2 weeks,I have been prepared for MORE than a year and ...
712,Los Angeles,More than 1 week,More than 1 week,"I am NOT prepared, and I do not intend to prep..."
713,Los Angeles,More than 1 week,More than 1 week,I have been prepared for MORE than a year and ...
714,San Bernardino,1 to 3 days,3 days to 1 week,I have been prepared for MORE than a year and ...


In [15]:
unique_values = {column: selected_fema_household_data[column].unique() for column in selected_fema_household_data.columns}
unique_values

{'What county in [state] do you live in? ': array(['Alameda', 'Sacramento', 'Orange', 'Los Angeles', 'San Diego',
        'Tulare', 'Humboldt', 'Alpine', 'Sonoma', 'Mendocino',
        'San Francisco', 'Santa Clara', 'San Bernardino', 'Marin',
        'Ventura', 'Merced', 'Fresno', 'El Dorado', 'San Mateo', 'Solano',
        'Yuba', 'Santa Barbara', 'Nevada', 'Inyo', 'Monterey',
        'San Joaquin', 'Contra Costa', 'Calaveras', 'Kern', 'Riverside',
        'Kings', 'Shasta', 'Butte', 'Stanislaus', 'Yolo', 'Madera',
        'Imperial', 'Tuolumne', 'Plumas', 'San Benito', 'Trinity',
        'Santa Cruz', 'Placer', 'Lassen', 'San Luis Obispo', 'Lake',
        'Siskiyou', 'Amador', 'Sutter'], dtype=object),
 'How long could you live in your home without power?': array(['3 days to 1 week', '1 to 3 days', "Don't know", 'Less than 1 day',
        'More than 3 months', 'More than 1 week', 'More than 1 month',
        'More than 2 weeks'], dtype=object),
 'How long could you live in your home

In [20]:
# Define the conditions
conditions = {
    'How long could you live in your home without power?': ['1 to 3 days', "Don't know", 'Less than 1 day'],
    'How long could you live in your home without running water? ': ["Don't know", 'Less than 1 day', '1 to 3 days'],
    'Thinking about preparing yourself for a disaster, which of the following best represents your degree of preparedness? ': [
        'I am NOT prepared, but I intend to get prepared in the next six months',
        'I am NOT prepared, and I do not intend to prepare in the next year',
        'I am NOT prepared, but I intend to start preparing in the next year',
        "Don't know"
    ]
}

# Create the new DataFrame with boolean values
boolean_df = pd.DataFrame()
boolean_df['What county in [state] do you live in? '] = selected_fema_household_data['What county in [state] do you live in? ']
for column, invalid_answers in conditions.items():
    boolean_df[column] = selected_fema_household_data[column].apply(lambda x: 0 if x in invalid_answers else 1)

# Retain the 'What county in [state] do you live in? ' column

boolean_df

,What county in [state] do you live in?,How long could you live in your home without power?,How long could you live in your home without running water?,"Thinking about preparing yourself for a disaster, which of the following best represents your degree of preparedness?"
0,Alameda,1,0,0
1,Sacramento,0,0,0
2,Orange,0,1,1
3,Los Angeles,1,1,0
4,Sacramento,1,0,0
...,...,...,...,...
711,Solano,1,1,1
712,Los Angeles,1,1,0
713,Los Angeles,1,1,1
714,San Bernardino,0,1,1


In [24]:
boolean_df = boolean_df.rename(columns={
    'What county in [state] do you live in? ':'county',
    'How long could you live in your home without power? ':'prepared_without_power',
    'How long could you live in your home without running water? ':'prepared_without_running_water',
    'Thinking about preparing yourself for a disaster, which of the following best represents your degree of preparedness? ':'general_preparedness'
})

In [42]:
boolean_df

,county,prepared_without_power,prepared_without_running_water,general_preparedness
0,Alameda,1,0,0
1,Sacramento,0,0,0
2,Orange,0,1,1
3,Los Angeles,1,1,0
4,Sacramento,1,0,0
...,...,...,...,...
711,Solano,1,1,1
712,Los Angeles,1,1,0
713,Los Angeles,1,1,1
714,San Bernardino,0,1,1


In [50]:
# Group by county, sum the boolean values, and count the occurrences of each county
grouped_df = boolean_df.groupby('county').agg({
    'prepared_without_power': 'sum',
    'prepared_without_running_water': 'sum',
    'general_preparedness': 'sum',
    'county': 'size'
}).rename(columns={'county': 'county_count'}).reset_index()

# Group by county, sum the boolean values, and count the occurrences of each county
grouped_df = boolean_df.groupby('county').agg({
    'prepared_without_power': 'sum',
    'prepared_without_running_water': 'sum',
    'general_preparedness': 'sum'
})

# Add 'county_count' column
grouped_df['county_count'] = boolean_df['county'].value_counts()

# Reset the index to move 'county' back as a column
grouped_df.reset_index(inplace=True)

grouped_df

,county,prepared_without_power,prepared_without_running_water,general_preparedness,county_count
0,Alameda,12,12,11,21
1,Alpine,0,0,1,1
2,Amador,2,2,1,2
3,Butte,3,3,3,5
4,Calaveras,5,7,6,7
5,Contra Costa,8,8,10,18
6,El Dorado,2,1,1,2
7,Fresno,11,8,8,21
8,Humboldt,3,2,2,3
9,Imperial,4,5,0,5


In [54]:
fema_household_metric = pd.DataFrame()
fema_household_metric['county'] = grouped_df['county']
fema_household_metric['county_count'] = grouped_df['county_count']
fema_household_metric['percent_prepared_without_power'] = (grouped_df['prepared_without_power'] / grouped_df['county_count']) * 100
fema_household_metric['percent_prepared_without_water'] = (grouped_df['prepared_without_running_water'] / grouped_df['county_count']) * 100
fema_household_metric['percent_prepared_for_general_disaster'] = (grouped_df['general_preparedness'] / grouped_df['county_count']) * 100

fema_household_metric

,county,county_count,percent_prepared_without_power,percent_prepared_without_water,percent_prepared_for_general_disaster
0,Alameda,21,57.142857,57.142857,52.380952
1,Alpine,1,0.000000,0.000000,100.000000
2,Amador,2,100.000000,100.000000,50.000000
3,Butte,5,60.000000,60.000000,60.000000
4,Calaveras,7,71.428571,100.000000,85.714286
5,Contra Costa,18,44.444444,44.444444,55.555556
6,El Dorado,2,100.000000,50.000000,50.000000
7,Fresno,21,52.380952,38.095238,38.095238
8,Humboldt,3,100.000000,66.666667,66.666667
9,Imperial,5,80.000000,100.000000,0.000000
